# 01 - Quantum Neural Networks

#### Uncomment the cell below to pip install the necessary modules if not already installed

#### Note: Works with Qiskit Version 1.4.1 and Quantum Rings Qiskit Toolkit Version 0.1.10

In [1]:
# %pip install qiskit-machine-learning
# %pip install qiskit==1.4.1
# %pip install quantumrings-toolkit-qiskit==0.1.10

#### Restart the kernel after installing any of the missing packages

#### Brief Introduction

The topic of Machine Learning and Neural Networks is vast and broad in both the classical and quantum sense of the subject. Therefore, we will keep this introduction as brief as possible just to cover the main points implemented in this notebook.

Classical neural networks, as one might assume, was heavily inspired by the human brain. It is an algorithmic approach that was developed based on our own ability towards pattern recognition and for solving complex problems. Similar to how children learn through repetition or by example, a neural network learns through a training process based on a specific set of data which contains patterns the network is designed to "understand". It is a layered structure of interconnected nodes, or neurons, that are given parameters which can be dynamically modified during the training process known as the weights. In the brain, these weights would be equivalent to the strength of the synapse connection between neurons. 

From a machine learning perspective, quantum neural networks, or quantum machine learning (QML) is motivated by the integration of classical neural networks and parameterized quantum circuits. The quantum neural network (QNN) is a similar algorithmic model that can be trained to find hidden patterns in data similar to the classical neural network. The models load classical data into a quantum state, and process it with quantum gates parameterized by trainable weights. The output of this state can be plugged into a loss function to train the weights through backpropagation.

From a quantum computing perspective, the QNN is just a quantum algorithm based on a parameterized quantum circuit that can be trained using variational methods for parameter adjustment via classical optimizers (Gradient Descent, Linear Programming, Simulated Annealing, etc). The circuit is a combination of smaller circuits, or layers, similar to classical neural networks. It will have at least one layer of a sub-circuit that is the feature map for the input parameters followed by a second layer for data processing with variational weights. These layers can be repeated as many times as desired and will always be followed by a measurement layer at the very end of the circuit.

As mentioned at the start, this is just a brief overview of both topics. There are numerous branches of both classical and quantum machine learning each with their own sub-branches into specific algorithms, approaches, applications, and so much more. For now we will stop here, and start dipping our toes into building a circuit to model a QNN using estimators and samplers.

If you want to know more, please check the link below for Qiskit specific information on QML

In [ ]:
# This code is at:
# https://qiskit-community.github.io/qiskit-machine-learning/tutorials/01_neural_networks.html

We set the seed to ensure that the results are consistent. For "random" results, seed with system time using the commented code in the cell. The algorithm_globals is a class for generating random values for any algorithm used when calling algorithm_globals.random. By setting a seed, we ensure repeatability. For fun, if you want something more random, comment out the answer to lifes biggest question and uncomment the import time and time.time(). This will give it a random seed based on the system time rather than a static seed.

In [ ]:
# import time

# algorithm_globals.random_seed = time.time()

from qiskit_machine_learning.utils import algorithm_globals

algorithm_globals.random_seed = 42 # The answer to lifes biggest question

### Step 1: Instantiate the Quantum Circuit: EstimatorQNN vs SamplerQNN
There are two ways we can approach the implementation of a QNN. We have an EstimatorQNN and a SamplerQNN. They are similar in their creation with some minor variations. Ultimately, their functionality differ in that the EstimatorQNN is based on the network evaluation of quantum mechanical observables. The SamplerQNN is based on the samples resulting from measuring a quantum circuit. These will be elaborated upon in their respective sections.
#### Step 1a: EstimatorQNN
The EstimatorQNN takes in a parameterized quantum circuit as input along with optional quantum mechanical observables, and outputs expectation value computations for the forward pass. We can also give the estimator a list of observables for more complex circuits. First we will construct a simple circuit with two parameters, one as the QNN input and the other as a trainable weight.


In [ ]:
from qiskit.circuit import Parameter
from qiskit import QuantumCircuit

params1 = [Parameter("input1"), Parameter("weight1")]
qc1 = QuantumCircuit(1)
qc1.h(0)
qc1.ry(params1[0], 0)
qc1.rx(params1[1], 0)
qc1.draw("mpl", style="clifford")

We create the observable below to define the expectation value computation. If this is not set, then the estimator will automatically create a default observable $Z^{\otimes n}$. Here, $n$ is the number of qubits in the quantum circuit. For the observable we create below, we set the observable as $Y^{\otimes n}$.

In [ ]:
from qiskit.quantum_info import SparsePauliOp

observable1 = SparsePauliOp.from_list([("Y" * qc1.num_qubits, 1)])

Now that we have our circuit and observable, we want to instantiate the EstimatorQNN. The constructor for this takes four key arguments:

- `estimator`: optional primitive instance
- `pass_manager`: optional pass_manager instance for primitives that require transpilation
- `input_params`: list of quantum circuit parameters that should be treated as "network inputs"
- `weight_params`: list of quantum circuit parameters that should be treated as "network weights"

Based on our definition above, the first parameter of params1 is our input_params, and the second parameter is our weight_params. We do not define an estimator, this will cause the EstimatorQNN to create an instance of the base Estimator. The pass_manager is only necessary if the circuit needs to be transpiled which is not necessary for our current purposes.

In [ ]:
# From the toolkit, we can implement the EstimatorQNN
from quantumrings.toolkit.qiskit.machine_learning import QrEstimatorQNN as EstimatorQNN

estimator_qnn = EstimatorQNN(
    circuit=qc1, observables=observable1, input_params=[params1[0]], weight_params=[params1[1]]
)
# This concludes the creation of our  EstimatorQNN. Next we will look at the SamplerQNN

#### Step 1b: SamplerQNN

The SamplerQNN is created in a similar manner as the EstimatorQNN. The major difference lies in the sampler consuming samples from measuring the quantum circuit. This means it does not require a custom observable. The output samples are interpreted by default as the probabilities of measuring the integer index corresponding to a bitstring. We can also specify an interpret function to post-process the samples. This function should be defined so that it takes a measured integer from a bitstring and maps it to a new value, i.e. non-negative integer

If a custom interpret function is defined, the output_shape cannot be inferred by the network and needs to be provided explicitly.

If no interpret function is used, the dimension of the probability vector scales exponentially with the number of qubits. With a custom interpret function, this scaling can change.

We will create a different quantum circuit for the SamplerQNN. We will have two input parameters and four trainable weights that parameterize a two-local circuit.

In [ ]:
from qiskit.circuit import ParameterVector

inputs2 = ParameterVector("input", 2)
weights2 = ParameterVector("weight", 4)
print(f"input parameters: {[str(item) for item in inputs2.params]}")
print(f"weight parameters: {[str(item) for item in weights2.params]}")

qc2 = QuantumCircuit(2)
qc2.ry(inputs2[0], 0)
qc2.ry(inputs2[1], 1)
qc2.cx(0, 1)
qc2.ry(weights2[0], 0)
qc2.ry(weights2[1], 1)
qc2.cx(0, 1)
qc2.ry(weights2[2], 0)
qc2.ry(weights2[3], 1)

qc2.draw("mpl", style="clifford")

In [ ]:
# From the toolkit, we can implement the SamplerQNN as we did the EstimatorQNN before. 
# The key parameters for the SamplerQNN are the same as the EstimatorQNN defined above.

from quantumrings.toolkit.qiskit.machine_learning import QrSamplerQNN as SamplerQNN

sampler_qnn = SamplerQNN(circuit=qc2, input_params=inputs2, weight_params=weights2)
sampler_qnn

### Step 2: Run a Forward Pass

In it's most basic form, a forward pass is just a run through the neural network. Each input is passed through each layer and produces an output. Here we create random inputs and weights for the estimator and sampler using the algorithm_globals which we seeded at the start. 

First, we will run both the estimator and the sampler in a non-batched form. This form means that we just give it all the inputs we have for the circuit. We will follow this up with a batched forward pass. Look at how the outputs and shapes of the returned values change.

In [ ]:
estimator_qnn_input = algorithm_globals.random.random(estimator_qnn.num_inputs)
estimator_qnn_weights = algorithm_globals.random.random(estimator_qnn.num_weights)

In [ ]:
print(
    f"Number of input features for EstimatorQNN: {estimator_qnn.num_inputs} \nInput: {estimator_qnn_input}"
)
print(
    f"Number of trainable weights for EstimatorQNN: {estimator_qnn.num_weights} \nWeights: {estimator_qnn_weights}"
)

In [ ]:
sampler_qnn_input = algorithm_globals.random.random(sampler_qnn.num_inputs)
sampler_qnn_weights = algorithm_globals.random.random(sampler_qnn.num_weights)

In [ ]:
print(
    f"Number of input features for SamplerQNN: {sampler_qnn.num_inputs} \nInput: {sampler_qnn_input}"
)
print(
    f"Number of trainable weights for SamplerQNN: {sampler_qnn.num_weights} \nWeights: {sampler_qnn_weights}"
)

In [ ]:
estimator_qnn_forward = estimator_qnn.forward(estimator_qnn_input, estimator_qnn_weights)

print(
    f"Forward pass result for EstimatorQNN: {estimator_qnn_forward}. \nShape: {estimator_qnn_forward.shape}"
)

In [ ]:
sampler_qnn_forward = sampler_qnn.forward(sampler_qnn_input, sampler_qnn_weights)

print(
    f"Forward pass result for SamplerQNN: {sampler_qnn_forward}.  \nShape: {sampler_qnn_forward.shape}"
)

#### Now we batch the inputs. This is done by passing in a list of the input values.

In [ ]:
estimator_qnn_forward_batched = estimator_qnn.forward(
    [estimator_qnn_input, estimator_qnn_input], estimator_qnn_weights
)

print(
    f"Forward pass result for EstimatorQNN: {estimator_qnn_forward_batched}.  \nShape: {estimator_qnn_forward_batched.shape}"
)

In [ ]:
sampler_qnn_forward_batched = sampler_qnn.forward(
    [sampler_qnn_input, sampler_qnn_input], sampler_qnn_weights
)

print(
    f"Forward pass result for SamplerQNN: {sampler_qnn_forward_batched}.  \nShape: {sampler_qnn_forward_batched.shape}"
)

#### While a forward pass may be good for some cases, sometimes we wish to know how the network evolves

By default, the backward pass uses gradient descent to look at how the weights change as the input is propagated through the network. You can enable gradients for the input parameters by setting the attribute input_gradients=True for each of the QNNs. We initially show the result without an input gradient, and then with an input gradient.

In [ ]:
estimator_qnn_input_grad, estimator_qnn_weight_grad = estimator_qnn.backward(
    estimator_qnn_input, estimator_qnn_weights
)

print(
    f"Input gradients for EstimatorQNN: {estimator_qnn_input_grad}.  \nShape: {estimator_qnn_input_grad}"
)
print(
    f"Weight gradients for EstimatorQNN: {estimator_qnn_weight_grad}.  \nShape: {estimator_qnn_weight_grad.shape}"
)

In [ ]:
sampler_qnn_input_grad, sampler_qnn_weight_grad = sampler_qnn.backward(
    sampler_qnn_input, sampler_qnn_weights
)

print(
    f"Input gradients for SamplerQNN: {sampler_qnn_input_grad}.  \nShape: {sampler_qnn_input_grad}"
)
print(
    f"Weight gradients for SamplerQNN: {sampler_qnn_weight_grad}.  \nShape: {sampler_qnn_weight_grad.shape}"
)

In [ ]:
estimator_qnn.input_gradients = True
sampler_qnn.input_gradients = True

In [ ]:
estimator_qnn_input_grad, estimator_qnn_weight_grad = estimator_qnn.backward(
    estimator_qnn_input, estimator_qnn_weights
)

print(
    f"Input gradients for EstimatorQNN: {estimator_qnn_input_grad}.  \nShape: {estimator_qnn_input_grad.shape}"
)
print(
    f"Weight gradients for EstimatorQNN: {estimator_qnn_weight_grad}.  \nShape: {estimator_qnn_weight_grad.shape}"
)

In [ ]:
sampler_qnn_input_grad, sampler_qnn_weight_grad = sampler_qnn.backward(
    sampler_qnn_input, sampler_qnn_weights
)

print(
    f"Input gradients for SamplerQNN: {sampler_qnn_input_grad}.  \nShape: {sampler_qnn_input_grad.shape}"
)
print(
    f"Weight gradients for SamplerQNN: {sampler_qnn_weight_grad}.  \nShape: {sampler_qnn_weight_grad.shape}"
)

### Step 3: Advanced Functionality: Multiple Observables with EstimatorQNN, Custom Interpret with SamplerQNN

For more complex QNN architectures, we can pass in a list of observables to the EstimatorQNN. Below we create a second observable for $Z^{\otimes n}$ to include in the estimator. We then carry out a forward and backward, non-batched run.

### Multiple Observables

In [ ]:
observable2 = SparsePauliOp.from_list([("Z" * qc1.num_qubits, 1)])

estimator_qnn2 = EstimatorQNN(
    circuit=qc1,
    observables=[observable1, observable2],
    input_params=[params1[0]],
    weight_params=[params1[1]],
)

In [ ]:
estimator_qnn_forward2 = estimator_qnn2.forward(estimator_qnn_input, estimator_qnn_weights)
estimator_qnn_input_grad2, estimator_qnn_weight_grad2 = estimator_qnn2.backward(
    estimator_qnn_input, estimator_qnn_weights
)

print(f"Forward output for EstimatorQNN1: {estimator_qnn_forward.shape}")
print(f"Forward output for EstimatorQNN2: {estimator_qnn_forward2.shape}")
print(f"Backward output for EstimatorQNN1: {estimator_qnn_weight_grad.shape}")
print(f"Backward output for EstimatorQNN2: {estimator_qnn_weight_grad2.shape}")

### Custom Interpreter

As mentioned above, we can give the SamplerQNN a custom interpreter for measuring the integer index of the bitstring. Here we use a lambda function to define our interpreter. This will use a parity method, which allows it to perform binary classification. This will modify the output shape of the forward and backward passes. Feel free to modify this cell to use the parity function and use just the default interpret. Think about the differences of both.

In [ ]:
parity = lambda x: "{:b}".format(x).count("1") % 2
output_shape = 2  # parity = 0, 1

sampler_qnn2 = SamplerQNN(
    circuit=qc2,
    input_params=inputs2,
    weight_params=weights2,
    interpret=parity,
    output_shape=output_shape,
)

In [ ]:
sampler_qnn_forward2 = sampler_qnn2.forward(sampler_qnn_input, sampler_qnn_weights)
sampler_qnn_input_grad2, sampler_qnn_weight_grad2 = sampler_qnn2.backward(
    sampler_qnn_input, sampler_qnn_weights
)

print(f"Forward output for SamplerQNN1: {sampler_qnn_forward.shape}")
print(f"Forward output for SamplerQNN2: {sampler_qnn_forward2.shape}")
print(f"Backward output for SamplerQNN1: {sampler_qnn_weight_grad.shape}")
print(f"Backward output for SamplerQNN2: {sampler_qnn_weight_grad2.shape}")

### Conclusion

Congratulations! You have completed the first steps towards the development of a QNN! While this may not have seemed like much, you have applied to neural network classes to a quantum circuit and taken the first steps towards understanding how these two methods differ. A major part of machine learning is understanding the first steps with regard to input data processing and expected output. These first steps were taken in this notebook, and you are free to go back and review what you did. You are also encouraged to play with the different inputs, weights, random seed, and explore situations of multiple observables along with custom interpreters! As mentioned, this is an vast field and is continuing to expand as more developments occur in the world of quantum computing and QML. 